In [133]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GlobalMaxPooling1D, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import data_helper

In [134]:
df_train = pd.read_csv("tweet_clean_train.csv")
df_validation = pd.read_csv("tweet_clean_validation.csv")
df_test = pd.read_csv("tweet_clean_test.csv")

In [135]:
X_train = df_train["text"]
y_train = df_train["target"]

X_validation = df_validation["text"]
y_validation = df_validation["target"]

X_test = df_test["text"]
y_test = df_test["target"]

In [136]:
phrase_len = X_train.apply(lambda p: len(p.split(' ')))
max_phrase_len = phrase_len.max()
print('max phrase len: {0}'.format(max_phrase_len))

max phrase len: 93


In [137]:
# Create Sequence
vocab_size = 8192
tokenizer = Tokenizer(num_words = vocab_size, filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~')
tokenizer.fit_on_texts(X_train)

In [138]:
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen = max_phrase_len)
# y_train = to_categorical(y_train)

In [140]:
BATCH_SIZE = 512
EPOCHS = 8

In [145]:
model_lstm = Sequential()

model_lstm.add(Embedding(input_dim = vocab_size, output_dim = 256, input_length = max_phrase_len))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(1, activation = 'softmax'))
model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [146]:
history = model_lstm.fit(
    X_train,
    y_train,
    validation_split = 0.1,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE
)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 72000 samples, validate on 8000 samples
Epoch 1/8
72000/72000 [==============================] - 556s 8ms/step - loss: -15.4704 - accuracy: 0.0000e+00 - val_loss: -14.7122 - val_accuracy: 0.0000e+00
Epoch 2/8
40960/72000 [================>.............] - ETA: 3:51 - loss: -15.5279 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [106]:
sequences_train = tokenizer.texts_to_sequences(df_train["text"])
sequences_validation = tokenizer.texts_to_sequences(df_validation["text"])
sequences_test = tokenizer.texts_to_sequences(df_test["text"])
# sequences_test = tokenizer.texts_to_sequences(df_test["text"])
# print(len(sequences_test))
# sequences = tokenizer.texts_to_sequences(df_test["text"])

In [107]:
X_train = pad_sequences(sequences_train, maxlen = 500)
X_validation = pad_sequences(sequences_validation, maxlen = 500)
X_test = pad_sequences(sequences_test, maxlen = 500)

In [108]:
print(len(X_train))
print(len(X_validation))
print(len(X_test))

80000
16000
498


In [110]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, 500, input_length = 500))
model.add(LSTM(50, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print(model.summary)
model.fit(X_train, y_train, validation_data = (X_validation, y_validation), epochs = 3, batch_size = 512)

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x141957350>>


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 80000 samples, validate on 16000 samples
Epoch 1/3
  512/80000 [..............................] - ETA: 1:22:09 - loss: 0.6798 - accuracy: 0.1504

KeyboardInterrupt: 